# Introduction 
This is a work in progress. Wazeasy package is still under-development

In [1]:
import pandas as pd
from datetime import timedelta
from datetime import datetime as dt
import dask.dataframe as dd
from wazeasy import utils
import matplotlib.pyplot as plt
from shapely import wkt
import geopandas as gpd

In [2]:
storage_options={"profile":"wb_576"}
main_path = 's3://wbg-waze/bq/AR/576buenosaires/jams/'

In [3]:
# ddf = utils.load_data_parquet(main_path, 2023, 11, storage_options)
# utils.handle_time(ddf, 'America/Buenos_Aires', parquet = True)

In [4]:
def get_argentine_working_dates(year = None, month = None, date_range = None):
    days = pd.read_excel('/home/sol/Documents/wb/ASA_intermediate_cities/waze/data/Dias habiles.xlsx', dtype={'Fecha': 'str'})
    days['Fecha'] = pd.to_datetime(days['Fecha'])
    days = days[days['Día Hábil']==1].copy()
    if month:
        days['year'] = days.Fecha.dt.year
        days['month'] = days.Fecha.dt.month
        working_days = days[(days['year']==year)&(days['month']==month)]['Fecha'].dt.date
    else:
        print('Todo')
    return working_days.tolist()

In [5]:
month = 11
tables_cities = []
tables_streets = []
# Todo:adding the cities might necesary and another layer of confidence
streets_names = ['Av. Córdoba', 'Av. Corrientes'] #Care must be paid since filtering by names might not be accurate
for year in [2019, 2023]:
    print(year)
    ddf = utils.load_data_parquet(main_path, year, month, storage_options)
    utils.handle_time(ddf, 'America/Buenos_Aires', parquet = True)
    working_days = get_argentine_working_dates(year, month)
    ddf = ddf[ddf['date'].isin(working_days)]
    ddf = ddf[ddf['city'].notnull()]
    table = utils.get_summary_statistics_city(ddf, year, working_days)
    utils.harmonize_data(table)
    tables_cities.append(table)

    table = utils.get_summary_statistics_street(ddf, streets_names, year, working_days)
    tables_streets.append(table)
    
tables_cities = pd.concat(tables_cities, axis = 1)
tables_cities.to_csv('./results/table_cities_ba.csv')

tables_streets = pd.concat(tables_streets, axis = 1)
tables_streets.to_csv('./results/tables_streets_ba.csv')

2019
2023


In [29]:
rows = ['number_of_jams', 'total_jam_length', 'total_jam_length_level_1', 'total_jam_length_level_2', 'total_jam_length_level_3', 
        'total_jam_length_level_4', 'tci']
columns = [2019, 2023]

for street in streets_names:
    print('Street: {}'.format(street))
    d = pd.DataFrame(index = rows, columns = columns)
    for row in rows:
        for col in columns:
            d.loc[row, col] = tables_streets.loc[street][row+str(col)]
    d['perc_change'] = (d[2023]-d[2019])/d[2019]
    display(d)

Street: Av. Córdoba


,2019,2023,perc_change
number_of_jams,2284.0,1605.0,-0.297285
total_jam_length,21766.994,16565.513,-0.238962
total_jam_length_level_1,199761.0,120304.0,-0.39776
total_jam_length_level_2,7001699.0,4270575.0,-0.390066
total_jam_length_level_3,12094489.0,10651928.0,-0.119274
total_jam_length_level_4,2442335.0,1481844.0,-0.393268
tci,1088349.7,788833.952381,-0.275202


Street: Av. Corrientes


,2019,2023,perc_change
number_of_jams,4036.0,4739.0,0.174182
total_jam_length,23946.514,19942.105,-0.167223
total_jam_length_level_1,1459703.0,1273795.0,-0.12736
total_jam_length_level_2,7378661.0,6476569.0,-0.122257
total_jam_length_level_3,13544277.0,10870366.0,-0.19742
total_jam_length_level_4,1029586.0,1158578.0,0.125285
tci,1197325.7,949624.047619,-0.206879


In [ ]:
rows = ['number_of_jams', 'total_jam_length', 'total_jam_length_level_1', 'total_jam_length_level_2', 'total_jam_length_level_3', 
        'total_jam_length_level_4', 'tci']
columns = [2019, 2023]

for street in streets_names:
    print('Street: {}'.format(street))
    d = pd.DataFrame(index = rows, columns = columns)
    for row in rows:
        for col in columns:
            d.loc[row, col] = tables_streets.loc[street][row+str(col)]
    d['perc_change'] = (d[2023]-d[2019])/d[2019]
    display(d)

In [40]:
for city in ['Palermo']:
    print('Street: {}'.format(city))
    d = pd.DataFrame(index = rows, columns = columns)
    for row in rows:
        for col in columns:
            d.loc[row, col] = tables_cities.loc[city][row+str(col)]
    d['perc_change'] = (d[2023]-d[2019])/d[2019]
    display(d)

Street: Palermo


,2019,2023,perc_change
number_of_jams,29839.0,36029.0,0.207447
total_jam_length,71891.165,81182.718,0.129245
total_jam_length_level_1,1375465.0,2763545.0,1.009171
total_jam_length_level_2,25100931.0,24640263.0,-0.018353
total_jam_length_level_3,39174620.0,44363410.0,0.132453
total_jam_length_level_4,5525526.0,8085597.0,0.463317
tci,3594558.25,3865843.714286,0.075471
